In [1]:
import selenium
import os
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import csv

In [2]:
class ChinaBiddingScraper:
    def __init__(self):
        # Initialize the WebDriver
        self.driver = webdriver.Chrome(executable_path=r"C:\Users\shash\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
        self.url = "https://ieg.worldbankgroup.org/data"

    def scrape_data(self):
        # Navigate to the website
        self.driver.get(self.url)

        # Wait for page to load (you might need to adjust the time)
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="bidding"]/div/div/div/div[1]/div/div/div/span[1]')))
        
        # Scrape data manually
        bidding_list = []
        elements = self.driver.find_elements(By.XPATH, '//*[@id="bidding"]/div/div/div/div[1]/div/div/div/span')

        
        for element in elements:
            button=self.driver.find_element(By.XPATH, '//*[@id="bidding"]/div[2]/div[3]/div/div[1]/div/div/div/span[2]/a')
            button.click()
            title = element.find_element(By.XPATH, '//div[3]/div/div[3]/ul/li/div[1]/a[1]').text.strip()
            link = element.find_element(By.XPATH, '//div[3]/div/div[3]/ul/li/div[1]/a[1]').get_attribute("href")
            date = element.find_element(By.XPATH, '//div[3]/div/div[3]/ul/li/div[1]/span').text.strip()
            
            bidding_data = {
                'Title': title,
                'Link': link,
                'Date': date,
            }
            bidding_list.append(bidding_data)

        return bidding_list

    def save_to_csv(self, data, filename):
        # Save the scraped data to a CSV file
        with open(filename, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['Title', 'Link', 'Date']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            writer.writeheader()
            for item in data:
                writer.writerow(item)

if __name__ == "__main__":
    scraper = ChinaBiddingScraper()
    scraped_data = scraper.scrape_data()

    if scraped_data:
        save_directory = r'C:\Users\shash\Downloads'
        save_filename = 'chinabidding_data.csv'
        save_path = os.path.join(save_directory, save_filename)

        try:
            scraper.save_to_csv(scraped_data, save_path)
            print(f"Data scraped and saved to '{save_path}'")
        except Exception as e:
            print(f"Error saving the CSV file: {e}")
        finally:
            # Ensure the WebDriver is properly closed
            scraper.driver.quit()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF722E152A2+57122]
	(No symbol) [0x00007FF722D8EA92]
	(No symbol) [0x00007FF722C5E3AB]
	(No symbol) [0x00007FF722C97D3E]
	(No symbol) [0x00007FF722C97E2C]
	(No symbol) [0x00007FF722CD0B67]
	(No symbol) [0x00007FF722CB701F]
	(No symbol) [0x00007FF722CCEB82]
	(No symbol) [0x00007FF722CB6DB3]
	(No symbol) [0x00007FF722C8D2B1]
	(No symbol) [0x00007FF722C8E494]
	GetHandleVerifier [0x00007FF7230BEF82+2849794]
	GetHandleVerifier [0x00007FF723111D24+3189156]
	GetHandleVerifier [0x00007FF72310ACAF+3160367]
	GetHandleVerifier [0x00007FF722EA6D06+653702]
	(No symbol) [0x00007FF722D9A208]
	(No symbol) [0x00007FF722D962C4]
	(No symbol) [0x00007FF722D963F6]
	(No symbol) [0x00007FF722D867A3]
	BaseThreadInitThunk [0x00007FF98F7526AD+29]
	RtlUserThreadStart [0x00007FF99130AA68+40]


In [3]:
import requests
import json
import os
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup

In [4]:
class Weather_API:
    
    def __init__ (self, keyword):
        self.keyword = keyword
        
    def json_print(self, obj):
        # create a formatted string of the Python JSON object
        with open('api_data.txt', 'w') as json_file:
            json.dump(obj, json_file)
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)
        
    def create_dataframe(self, obj):
        
        # creating a dataframe from nested JSON objects
        FIELDS = ["source.id", "source.name", "author", "title", "description", "url", "urlToImage", "publishedAt", "content"]
        df = pd.json_normalize(obj['articles'])
        final_df = df[FIELDS]
        #final_df.set_index('source.id', inplace = True)
        display(final_df.head())

    
    def news_api(self):
        
        # Use the news-api to obtain articles published from
        url = ('https://newsapi.org/v2/everything?'
       'q={keyword}&'
       'apiKey=4e70cabb80884db08524a28ac33cdc1d'.format(keyword = self.keyword))
        
        
        response = requests.get(url)
        if (response.status_code == 200):
            print('API call successful!')
            json_response = response.json()
            if(len(json_response['articles']) == 0):
                print('No News Articles Found')
            else:
                
                # Print a String in Json Format
                self.json_print(json_response)
                
                
                
                # Create a pandas DataFrame
                self.create_dataframe(json_response)
                    
        else:
            print('Status code: ', response.status_code)

In [5]:
class Web_Scraping:
    
    def __init__(self, location):
        self.location = location
        
    def selenium_webdriver(self):
        
        # Start the Driver
        driver = webdriver.Chrome(executable_path = r"C:\Users\shash\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
        
    
        url = ('https://earthdata.nasa.gov/search?q={location}'.format(location = self.location))
        driver.get(url)
        time.sleep(15)
        
        # Driver scrolls down 25 times to load the table.
        for i in range(0,30):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(10)
            
        # Fetch the webpage and store in a variable.
        webpage = driver.page_source
        
        # Parse the page using BeautifulSoup
        HTMLPage = BeautifulSoup(webpage, 'html.parser')
        
        titles = []
        description = []
        links = []

        for lists in HTMLPage.find_all(class_ = 'result'):
            if (lists.span.text != '' and len(lists.find_all('p')) != 0):
                titles.append(lists.span.text)
                description.append(lists.find('p', class_ = '').text)
                links.append(lists.find('p', class_ = 'search-link').text)
        
        # Create a DataFrame
        df = pd.DataFrame(list(zip(titles, description, links)),
               columns =['title', 'description', 'link'])
        
        display(df)
        
        # Store to csv file
        df.to_csv('ws.csv', sep=',', index=False,header=True)

In [6]:
keyword = input('Enter Keyword to be searched: ').lower()
w_api = Weather_API(keyword)
w_api.news_api()

location = input('Enter Location: ').lower()
ws = Web_Scraping('India')
ws.selenium_webdriver()

Enter Keyword to be searched: Delhi
API call successful!
{
    "articles": [
        {
            "author": null,
            "content": "World leaders are arriving in the Indian capital Delhi this week for the G20 summit, and authorities have gone to great lengths to spruce up the city.\r\nThey have undertaken elaborate beautification d\u2026 [+513 chars]",
            "description": "India's capital Delhi has undergone a makeover as it prepares to host world leaders.",
            "publishedAt": "2023-09-07T10:30:40Z",
            "source": {
                "id": "bbc-news",
                "name": "BBC News"
            },
            "title": "G20 summit: What do the people of Delhi think about the event?",
            "url": "https://www.bbc.co.uk/news/av/world-asia-india-66740328",
            "urlToImage": "https://ichef.bbci.co.uk/news/1024/branded_news/6FB9/production/_131010682_p0gc58w3.jpg"
        },
        {
            "author": "https://www.facebook.com/bbcnews",
    

,source.id,source.name,author,title,description,url,urlToImage,publishedAt,content
0,bbc-news,BBC News,None,G20 summit: What do the people of Delhi think ...,India's capital Delhi has undergone a makeover...,https://www.bbc.co.uk/news/av/world-asia-india...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-09-07T10:30:40Z,World leaders are arriving in the Indian capit...
1,bbc-news,BBC News,https://www.facebook.com/bbcnews,In pictures: India gets ready for G20,The summit is just a day away and Indian capit...,https://www.bbc.co.uk/news/world-asia-india-66...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-09-08T05:05:00Z,India is poised to host the prestigious G20 su...
2,None,Yahoo Entertainment,Reuters,India will buy oil from anyone who offers the ...,NEW DELHI (Reuters) -India will buy oil from a...,https://sg.finance.yahoo.com/news/india-buy-oi...,https://media.zenfs.com/en/reuters.com/a8056d7...,2023-08-30T05:56:19Z,NEW DELHI (Reuters) -India will buy oil from a...
3,bbc-news,BBC News,https://www.facebook.com/bbcnews,Biden disappointed Xi will not attend G20 summit,The two leaders last met on the side-lines of ...,https://www.bbc.co.uk/news/world-asia-china-66...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-09-04T02:21:48Z,"US President Joe Biden has said he is ""disappo..."
4,time,Time,Jenny Leonard / Bloomberg,First Lady Jill Biden Tests Positive for COVID...,The President is slated to attend the G-20 whi...,https://time.com/6310590/jill-biden-covid-19-p...,https://api.time.com/wp-content/uploads/2023/0...,2023-09-05T02:40:00Z,U.S. First Lady Jill Biden has tested positive...


Enter Location: India


,title,description,link
